In [13]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1")


In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})

print(prompt)

ai_message = llm.invoke(prompt_template.invoke({
    "country": "France"
}))

print(ai_message)

output_parser = StrOutputParser()

answer = output_parser.invoke(
    llm.invoke(prompt_template.invoke({
        "country": "France"
    }))
)

text='What is the capital of France? Return the name of the city only'
content='Paris' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-07-31T14:23:25.385242Z', 'done': True, 'done_reason': 'stop', 'total_duration': 111771800, 'load_duration': 24207300, 'prompt_eval_count': 24, 'prompt_eval_duration': 1020700, 'eval_count': 2, 'eval_duration': 85596300, 'model_name': 'llama3.1'} id='run--1844e541-2c20-485e-a17e-5a5cd0d41b59-0' usage_metadata={'input_tokens': 24, 'output_tokens': 2, 'total_tokens': 26}


In [15]:
capital_chain = prompt_template | llm | output_parser

llm은 runnable이기 때문에 파이프로 연결한다 => 모든 모델이 runnable 클래스를 상속받음

In [16]:
capital_chain.invoke({
    "country": "France"
})

'Paris'

In [23]:
country_prompt = PromptTemplate(
    template="""Guess the name of the {continent} based on the following information:
    {information}
    Returnm the name of the coutnry only
    """,
    input_variables=["information"],
)
country_chain = country_prompt | llm | output_parser
# country_chain.invoke({
#     "information": "This country is very famous for its wine in Europe"
# })

In [19]:
final_chain = {"country": country_chain} | capital_chain

In [20]:
final_chain.invoke({
    "information": "This country is very famous for its wine in Europe"
})

'Rome'

In [22]:
# Runnable pass through => 전달되는 runnable
# runnbale pass through를 사용하면 다음과 같이 체인 사용이 가능하다.
from langchain_core.runnables import RunnablePassthrough

final_chain = {
    "information": RunnablePassthrough()
} | {
    "country": country_chain
} | capital_chain

final_chain.invoke("This country is very famous for its wine in Europe") # 이 정보가 information으로 들어감 => 이 결과가 country가 되어 capital chain의 input(invoke)로 들어감

'Paris'

In [24]:
# Runnable pass through => 전달되는 runnable
# runnbale pass through를 사용하면 다음과 같이 체인 사용이 가능하다.
from langchain_core.runnables import RunnablePassthrough

final_chain = {
                  "information": RunnablePassthrough(),
    "country": RunnablePassthrough()
              } | {
                  "country": country_chain
              } | capital_chain

final_chain.invoke({
    "information": "This country is very famous for its wine in Europe",
    "continent": "Europe"
}) # RunnablePassThrough를 2개 이상 사용할 경우 invoke의 input은 key value 형태로 작성해주어야 함

'Paris'